# import

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 326.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f9d453c6a73f0e1504cd93bf0c010c0fda2cace2da27549a7d79db2432aef632
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
import gc
import json
import os
import math

import cupy as cp
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import joblib
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from cuml.metrics import pairwise_distances
from cuml.neighbors import NearestNeighbors
from cuml.svm import SVC
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import WandbLogger
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import train_test_split
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)
from torch.utils.data import DataLoader
from torch.nn import Sigmoid

from tqdm import tqdm
tqdm.pandas()

from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from sentence_transformers import InputExample
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator

# Config

In [3]:
cfg = {
    "general": {
        #"project_name": "LECR",
        "input_path": "/kaggle/input",
        "output_path": "./output",
        "seed": 42,
        "fold": [0],
        "cv": True,
    },
    
    "task": {
        #"train_select_top_n_pos": 100,
        "train_select_top_n_neg": 100,
        "valid_select_top_n": 50,
        "undersampling": True,
    },
    
    "cross_encoder": {
        "model_name": "/kaggle/input/lecr-retriever-fold0-model/english",
        "max_length": 256,
        "batch_size": 64,
        "gradient_checkpointing": False,
        "warmup_ratio": 0.1,
        "epoch": 5,
        "lr": 5e-5,
        "weight_decay": 1e-2,
        "freeze_layers": 0,
        "reinit_layers": 0,
    },
}

## Read Data

In [4]:
correlations_df = pd.read_csv(f"{cfg['general']['input_path']}/learning-equality-curriculum-recommendations/correlations.csv")

# Function

In [5]:
def get_positive_corr(fold_n):
    content_df = pd.read_pickle(f"{cfg['general']['input_path']}/lecr-datapkl/content.pkl")
    topics_df = pd.read_pickle(f"{cfg['general']['input_path']}/lecr-datapkl/topics.pkl")
    fold = pd.read_pickle(f"{cfg['general']['input_path']}/lecr-datapkl/fold.pkl")
    
    topics_df_fold = topics_df.merge(fold, how="inner", left_on="id", right_on="topic_id")
    topics_df_fold = topics_df_fold.drop(["topic_id"], axis=1)
    topics_df_fold = topics_df_fold[topics_df_fold["fold"]!=fold_n]

    topics_df_fold.rename(columns=lambda x: "topic_" + x, inplace=True)
    content_df.rename(columns=lambda x: "content_" + x, inplace=True)

    correlations = correlations_df.copy()
    correlations.content_ids = correlations.content_ids.str.split()
    correlations = correlations.explode("content_ids").rename(columns={"content_ids": "content_id"})
    correlations = correlations.merge(topics_df_fold, how="inner", on="topic_id")
    correlations = correlations.merge(content_df, how="inner", on="content_id")
    
    correlations.rename(columns={"content_content_inputs" : "content_inputs"}, inplace=True)
    correlations.rename(columns={"topic_topic_inputs" : "topic_inputs"}, inplace=True)
    
    return correlations

In [6]:
def get_score_using_threshold(x_val, val_predictions, correlations):
    best_score = 0
    best_threshold = None
    for thres in tqdm(np.arange(0.0, 1.0, 0.01)):
        x_val["predictions"] = np.where(val_predictions > thres, 1, 0)
        x_val1 = x_val[x_val["predictions"] == 1]
        x_val1 = x_val1.groupby(["topic_ids"])["content_ids"].unique().reset_index()
        x_val1["content_ids"] = x_val1["content_ids"].apply(lambda x: ' '.join(x))
        x_val1.columns = ["topic_id", "predictions"]
        x_val0 = pd.Series(x_val["topic_ids"].unique())
        x_val0 = x_val0[~x_val0.isin(x_val1["topic_id"])]
        x_val0 = pd.DataFrame({"topic_id": x_val0.values, "predictions": ""})
        x_val_r = pd.concat([x_val1, x_val0], axis = 0, ignore_index = True)
        x_val_r = x_val_r.merge(correlations, how = "left", on = "topic_id")
        score = f2_score(x_val_r["content_ids"], x_val_r["predictions"])
        if score > best_score:
            best_score = score
            best_threshold = thres
    return best_score, best_threshold

def get_score(x_val, val_predictions, correlations):
    #x_val["predictions"] = np.where(val_predictions > thres, 1, 0)
    x_val["predictions"] = val_predictions
    x_val1 = x_val[x_val["predictions"] == 1]
    x_val1 = x_val1.groupby(["topic_ids"])["content_ids"].unique().reset_index()
    x_val1["content_ids"] = x_val1["content_ids"].apply(lambda x: ' '.join(x))
    x_val1.columns = ["topic_id", "predictions"]
    x_val0 = pd.Series(x_val["topic_ids"].unique())
    x_val0 = x_val0[~x_val0.isin(x_val1["topic_id"])]
    x_val0 = pd.DataFrame({"topic_id": x_val0.values, "predictions": ""})
    x_val_r = pd.concat([x_val1, x_val0], axis = 0, ignore_index = True)
    x_val_r = x_val_r.merge(correlations, how = "left", on = "topic_id")
    score = f2_score(x_val_r["content_ids"], x_val_r["predictions"])
    return score

In [7]:
def reinit_layer(model):
    print(f"reinit layer -> {cfg['cross_encoder']['reinit_layers']}")
    for layer in model.encoder.layer[
        -cfg["cross_encoder"]["reinit_layers"] :
    ]:
        for module in layer.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(
                    mean=0.0, std=model.config.initializer_range
                )
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(
                    mean=0.0, std=model.config.initializer_range
                )
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)

In [8]:
def make_inputs(df):
    inputs = []
    
    for content_input, topic_input, target in df[["content_inputs", "topic_inputs", "target"]].values:
        inputs.append(
            InputExample(texts=[topic_input, content_input], label=target),
        )
    
    return inputs

In [9]:
class BalancedBatchUnderSampler(torch.utils.data.sampler.Sampler):
    def __init__(
        self, labels, batch_size,
    ):
        self.labels = labels

        label_counts = np.bincount(labels)
        major_label = label_counts.argmax()
        minor_label = label_counts.argmin()

        self.major_indices = np.where(labels == major_label)[0]
        self.minor_indices = np.where(labels == minor_label)[0]

        np.random.shuffle(self.major_indices)
        np.random.shuffle(self.minor_indices)

        self.used_indices = 0
        self.count = 0
        self.batch_size = batch_size
        self.n_samples = batch_size // 2

    def __iter__(self):
        np.random.shuffle(self.major_indices)
        np.random.shuffle(self.minor_indices)
        self.count = 0
        self.used_indices = 0
        while self.count + self.n_samples <= len(self.minor_indices):
            indices = (
                self.minor_indices[
                    self.used_indices : self.used_indices + self.n_samples
                ].tolist()
                + np.random.choice(
                    self.major_indices, self.n_samples, replace=False
                ).tolist()
            )
            np.random.shuffle(indices)
            yield indices

            self.used_indices += self.n_samples
            self.count += self.n_samples

    def __len__(self):
        return len(self.minor_indices) // self.n_samples

In [10]:
def train_and_predict(
    cfg,
    fold,
    train,
    valid,
):  
    # under sampling
    #rus = RandomUnderSampler(random_state=cfg["general"]["seed"])
    #train_X, train_y = train_X.reshape(-1, 1), train_y.reshape(-1, 1)
    #train, _ = rus.fit_resample(train, train["target"])
    #train_X, train_y = train_X.flatten(), train_y.flatten()
    
    train_inputs = make_inputs(train)
    valid_inputs = make_inputs(valid)
    
    if cfg["task"]["undersampling"]:
        batch_sampler = BalancedBatchUnderSampler(train["target"].values, cfg["cross_encoder"]["batch_size"])
        train_dataloader = DataLoader(train_inputs, batch_sampler=batch_sampler)
    else:
        train_dataloader = DataLoader(train_inputs, shuffle=True, batch_size=cfg["cross_encoder"]["batch_size"])
        
    evaluator = CEBinaryClassificationEvaluator.from_input_examples(valid_inputs, show_progress_bar=True)
    
    warmup_steps = math.ceil(len(train_dataloader) * cfg["cross_encoder"]["epoch"] * 0.1)
    model = CrossEncoder(cfg["cross_encoder"]["model_name"], num_labels=1, max_length=cfg["cross_encoder"]["max_length"])
    tokens = ["[T_SEP]", "[C_SEP]"]
    model.tokenizer.add_tokens(tokens, special_tokens=True)
    model.model.resize_token_embeddings(len(model.tokenizer))
    
    # freeze
    if cfg["cross_encoder"]["freeze_layers"] > 0:
        print(f"freeze layer -> {cfg['cross_encoder']['freeze_layers']}")
        model.model.base_model.embeddings.requires_grad_(False)
        model.model.base_model.encoder.layer[:cfg["cross_encoder"]["freeze_layers"]].requires_grad_(False)
    
    # reinit some layers
    if cfg["cross_encoder"]["reinit_layers"] > 0:
        reinit_layer(model.model.base_model)
    
    model.fit(
        train_dataloader=train_dataloader,
        evaluator=evaluator,
        epochs=cfg["cross_encoder"]["epoch"],
        warmup_steps=warmup_steps,
        output_path=f"{cfg['general']['output_path']}/fold{fold}/",
        use_amp=True,
        save_best_model=False,
    )
    model.save(f"{cfg['general']['output_path']}/crossencoder-model_fold{fold}")

    if valid is None:
        return
    else:
        predictions = model.predict(
            sentences=valid[["topic_inputs", "content_inputs"]].values,
            activation_fct=Sigmoid(),
            batch_size=cfg["cross_encoder"]["batch_size"],
        )
        return predictions

In [11]:
def one_fold(cfg, fold_n):
    print(f"[fold_{fold_n}] start")
    seed_everything(cfg["general"]["seed"], workers=True)
    
    train_neg = pd.read_pickle(f"/kaggle/input/lecr-knn-output-fold{fold_n}/train_fold_k{cfg['task']['train_select_top_n_neg']}_{fold_n}.pkl")
    train_neg = train_neg[train_neg["language"]=="en"]
    train_pos = get_positive_corr(fold_n)
    train_pos = train_pos[train_pos["topic_language"]=="en"]
    train_pos = train_pos[train_pos["content_language"]=="en"]
    train_pos["target"] = 1
    train_neg = train_neg[train_neg["target"]==0].reset_index(drop=True)
    train = pd.concat([train_pos, train_neg])
    del train_pos, train_neg
    gc.collect()
    print("read train data")
    valid = pd.read_pickle(f"/kaggle/input/lecr-knn-output-fold{fold_n}/valid_fold_k{cfg['task']['valid_select_top_n']}_{fold_n}.pkl")
    valid = valid[valid["language"]=="en"]
    print("read valid data")
    
    print("train_shape:", train.shape)
    print(train["target"].value_counts())
    print("valid_shape:", valid.shape)
    print(valid["target"].value_counts())

    # train and valid
    valid_preds = train_and_predict(
        cfg,
        fold_n,
        train,
        valid,
    )
    print(valid_preds)
    
    # metrics
    score, threshold = get_score_using_threshold(valid, valid_preds, correlations_df)
    print(f"Score: {score}")
    print(f"Threshold: {threshold}")
    valid_preds = np.where(valid_preds > threshold, 1, 0)
    accuracy = accuracy_score(valid["target"].values, valid_preds)
    recall = recall_score(valid["target"].values, valid_preds)
    specificity = recall_score(valid["target"].values, valid_preds, pos_label=0)
    precision = precision_score(valid["target"].values, valid_preds)
    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"Specificity: {specificity}")
    print(f"Precision: {precision}")
    results = pd.read_csv(f"{cfg['general']['output_path']}/fold{fold_n}/CEBinaryClassificationEvaluator_results.csv").drop(columns=["steps"])
    display(results)

    torch.cuda.empty_cache()

    return score, threshold, accuracy, recall, specificity, precision

In [12]:
def all_train(cfg, train):
    print("[all_train] start")
    seed_everything(cfg["general"]["seed"], workers=True)
    
    # train
    train_and_predict(cfg, train)

    return

In [13]:
def f2_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    tp = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    fp = np.array([len(x[1] - x[0]) for x in zip(y_true, y_pred)])
    fn = np.array([len(x[0] - x[1]) for x in zip(y_true, y_pred)])
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f2 = tp / (tp + 0.2 * fp + 0.8 * fn)
    return round(f2.mean(), 4)

def get_pos_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    int_true = np.array([len(x[0] & x[1]) / len(x[0]) for x in zip(y_true, y_pred)])
    return round(np.mean(int_true), 5)

## Main

In [14]:
# random seed setting
seed_everything(cfg["general"]["seed"], workers=True)

if cfg["general"]["cv"]:
    score_list = []
    threshold_list = []
    accuracy_list = []
    recall_list = []
    specificity_list = []
    predicision_list = []
    for fold_n in tqdm(cfg["general"]["fold"]):
        cfg["fold_n"] = fold_n
        score, threshold, accuracy, recall, specificity, predicision = one_fold(cfg, fold_n)
        score_list.append(score)
        threshold_list.append(threshold)
        accuracy_list.append(accuracy)
        recall_list.append(recall)
        specificity_list.append(specificity)
        predicision_list.append(predicision)

    score_mean = np.mean(score_list, axis=0)
    threshold_mean = np.mean(threshold_list, axis=0)
    accuracy_mean = np.mean(accuracy_list, axis=0)
    recall_mean = np.mean(recall_list, axis=0)
    specificity_mean = np.mean(specificity_list, axis=0)
    predicision_mean = np.mean(predicision_list, axis=0)
    print(f"cv mean score:{score_mean}")
    print(f"cv mean threshold:{threshold_mean}")
    print(f"cv mean accuracy:{accuracy_mean}")
    print(f"cv mean recall:{recall_mean}")
    print(f"cv mean specificity:{specificity_mean}")
    print(f"cv mean predicision:{predicision_mean}")
else:
    # train all data
    cfg["fold_n"] = "all"
    all_train(cfg, train_X, train_y)

  0%|          | 0/1 [00:00<?, ?it/s]

[fold_0] start
read train data
read valid data
train_shape: (2441204, 26)
0    2327006
1     114198
Name: target, dtype: int64
valid_shape: (184050, 7)
0    173519
1     10531
Name: target, dtype: int64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/lecr-retriever-fold0-model/english and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3568 [00:00<?, ?it/s]

...


  1%|          | 1/100 [00:00<00:44,  2.24it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys

100%|██████████| 100/100 [00:23<00:00,  4.23it/s]


Score: 0.4921
Threshold: 0.61
Accuracy: 0.9004020646563434
Recall: 0.6587218687683981
Specificity: 0.9150698194434039
Precision: 0.3200609024637815


,epoch,Accuracy,Accuracy_Threshold,F1,F1_Threshold,Precision,Recall,Average_Precision
0,0,0.950345,0.964407,0.452121,0.844487,0.425578,0.482195,0.441652
1,1,0.952209,0.969425,0.471452,0.812748,0.473256,0.469661,0.469508
2,2,0.951405,0.969085,0.471946,0.867828,0.472238,0.471655,0.461807
3,3,0.952431,0.972367,0.488841,0.877926,0.502052,0.476308,0.490312
4,4,0.952274,0.966925,0.493633,0.873898,0.490395,0.496914,0.492415


100%|██████████| 1/1 [4:27:58<00:00, 16078.42s/it]

cv mean score:0.4921
cv mean threshold:0.61
cv mean accuracy:0.9004020646563434
cv mean recall:0.6587218687683981
cv mean specificity:0.9150698194434039
cv mean predicision:0.3200609024637815


In [15]:
torch.cuda.empty_cache()
gc.collect()

45